In [2]:
!pip install transformers datasets tensorflow tf-keras

In [3]:
import os
import tensorflow as tf
from datasets import load_dataset
from transformers import TFAutoModel, AutoTokenizer

In [4]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [6]:
dataset = load_dataset('tyqiangz/multilingual-sentiments','all')

Generating train split:   0%|          | 0/270399 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10857 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14465 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 270399
    })
    validation: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 10857
    })
    test: Dataset({
        features: ['text', 'source', 'language', 'label'],
        num_rows: 14465
    })
})

In [8]:
def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)

In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=None)

Map:   0%|          | 0/270399 [00:00<?, ? examples/s]

Map:   0%|          | 0/10857 [00:00<?, ? examples/s]

Map:   0%|          | 0/14465 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset.set_format('tf', columns=['input_ids', 'attention_mask', 'label'])

In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 270399
    })
    validation: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10857
    })
    test: Dataset({
        features: ['text', 'source', 'language', 'label', 'input_ids', 'attention_mask'],
        num_rows: 14465
    })
})

In [12]:
BATCH_SIZE = 1

In [29]:
# test
example = tokenized_dataset['train'][0]
data = list(example.values())

In [30]:
print("Input IDs:", data[1])

Input IDs: tf.Tensor(
[    0   119 60282   280  1991 16392    66   123 29133 91440     2     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1], shape=(128,), dtype=int64)


In [31]:
print("Attention Mask:", data[2])

Attention Mask: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(128,), dtype=int64)


In [32]:
print("Label:", data[0])

Label: tf.Tensor(0, shape=(), dtype=int64)


In [33]:
def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2]
    }, data[0]

In [34]:
train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['train'][:])
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['validation'][:])
validation_dataset = validation_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [36]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(1, 128), dtype=int64, numpy=
array([[     0, 129374,   5624,   2121,      5,   8264,     91,  10727,
         58319,      6,  42955,   6178,  78107,    138,   2651,      5,
          3291,   1603,   3510,     40,   1506,      5,    159,   5125,
         38267,    373,  18484,      5,      5,    120,   6795,     21,
         24552,     19,   1847,  12340,      5,      5,   1733,    304,
             6,  38988,    254,    269,      2,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,
  

In [37]:
class XLMRobertaForClassification(tf.keras.Model):
    def __init__(self, roberta_model, num_classes, dropout_rate=0.3):
        super().__init__()
        self.roberta = roberta_model
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.roberta(inputs)[1]
        x = self.dropout(x)
        return self.fc(x)

In [38]:
classifier = XLMRobertaForClassification(model, num_classes=3, dropout_rate=0.3)

In [39]:
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [40]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
history = classifier.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,
    callbacks=[early_stopping]
)

In [ ]:
evaluation_results = classifier.evaluate(test_dataset)

In [ ]:
base_save_path = os.path.expanduser('~/my_models/')

In [ ]:
model_save_path_tf = os.path.join(base_save_path, 'saved_model_second')
classifier.save(model_save_path_tf, save_format='tf')

In [ ]:
tokenizer_save_path = os.path.join(base_save_path, 'saved_tokenizer')
tokenizer.save_pretrained(tokenizer_save_path)

In [ ]:
results_save_path = os.path.join(base_save_path, 'evaluation_results.txt')
with open(results_save_path, 'w') as f:
    f.write(f"Loss: {evaluation_results[0]}\n")
    f.write(f"Accuracy: {evaluation_results[1]}\n")

In [ ]:
model_save_path = os.path.join(base_save_path, 'saved_model')
classifier.save(model_save_path)